In [ ]:
!gdown --id 1kFyKea_Tk3mXyyTjyHuXdAstx9Aac03T

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kFyKea_Tk3mXyyTjyHuXdAstx9Aac03T
To: /content/data2.json
100% 270k/270k [00:00<00:00, 105MB/s]


In [ ]:
!pip install git+https://github.com/JohnGiorgi/seq2rel.git

  Cloning https://github.com/JohnGiorgi/seq2rel.git to /tmp/pip-req-build-iiwirpux
  Running command git clone --filter=blob:none --quiet https://github.com/JohnGiorgi/seq2rel.git /tmp/pip-req-build-iiwirpux
  Resolved https://github.com/JohnGiorgi/seq2rel.git to commit 6bcffb47da157e4139f3537372f6845a8b35d1a6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.5/464.5 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall en-core-web-sm==3.7.1

Found existing installation: en-core-web-sm 3.7.1
Uninstalling en-core-web-sm-3.7.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/en_core_web_sm-3.7.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/en_core_web_sm/*
Proceed (Y/n)? y
  Successfully uninstalled en-core-web-sm-3.7.1


In [ ]:
!python -m spacy download en_core_web_sm

2024-05-19 20:47:10.955088: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 20:47:10.955152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 20:47:11.064159: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-19 20:47:11.278528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 20:47:13.013197: W tensorflow/comp

In [ ]:
import spacy
import json
from seq2rel import Seq2Rel
from seq2rel.common import util
from tqdm import tqdm

In [ ]:
nlp = spacy.load('en_core_web_sm')

MAX_SPLIT_LENGHT = 450

def split_text_words(text):
    cleaned_text = text
    tokens = nlp(cleaned_text)
    if len(tokens)<=MAX_SPLIT_LENGHT:
        return [tokens.text]
    splits = []

    start_index = 0
    end_index = MAX_SPLIT_LENGHT-1
    while end_index!=len(tokens)-1:
        if tokens[end_index].text not in ['.','?','!']:
            while tokens[end_index].text not in ['.','?','!']:
                end_index-=1
        split = tokens[start_index:end_index+1]
        splits.append(split.text.strip())
        start_index = end_index+1
        end_index = start_index+MAX_SPLIT_LENGHT-1 if (start_index+MAX_SPLIT_LENGHT-1<=len(tokens)-1) else len(tokens)-1
    return splits

In [ ]:
pretrained_model = "docred"
seq2rel = Seq2Rel(pretrained_model,cuda_device=0)

with open('data2.json') as f:
  data = json.load(f)


processed_data = []

for article in tqdm(data):
  if article['url']!='https://www.nytimes.com/2023/10/14/world/europe/liverpool-ian-graham-data.html':
    continue
  text = article['text']
  splits = split_text_words(text)
  output = seq2rel(splits)
  extract_relations = util.extract_relations(output)
  for rel in extract_relations:
      for key, value in rel.items():
          for pair in value:
              processed_data.append({"subj": pair[0], "pred": key, "obj": pair[1]})


#print(len(processed_data))
print(processed_data)
with open('extracted_triples.json','w') as f:
  json.dump(processed_data, f)

downloading: 100%|##########| 419M/419M [00:22<00:00, 19.1MiB/s]


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/allennlp/common/cached_transformers.py:81: UserWarning: You specified 'reinit_modules' in allennlp.common.cached_transformers.get(), but 'load_weights' is set to False, so 'reinit_modules' will be ignored.
  warnings.warn(
100%|██████████| 169/169 [00:04<00:00, 41.31it/s]

[{'subj': (('ian graham', 'graham'), 'PER'), 'pred': 'EMPLOYER', 'obj': (('liverpool',), 'ORG')}, {'subj': (('champions league',), 'MISC'), 'pred': 'LEAGUE', 'obj': (('liverpool',), 'ORG')}]
